Following [this](https://pytorch-lightning.readthedocs.io/en/stable/debug/debugging_basic.html)


### How does Lightning help me debug ?

The Lightning `Trainer` has a lot of arguments devoted to maximizing your debugging productivity.

### Set a breakpoint

A breakpoint stops your code execution so you can inspect variables, etc… and allow your code to execute one line at a time.

In this example, the code will stop before executing the `y = x**2` line.

In [2]:
# Enter, when prompted by pdb:
# print(y)
# print(x)
# continue
# 

def function_to_debug():
    x = 2

    # set breakpoint
    import pdb

    pdb.set_trace()
    y = x ** 2

function_to_debug()

> /tmp/ipykernel_15866/1252560061.py(8)function_to_debug()
      6 
      7     pdb.set_trace()
----> 8     y = x ** 2
      9 
     10 function_to_debug()

*** NameError: name 'y' is not defined
2


### Run all your model code once quickly

If you’ve ever trained a model for days only to crash during validation or testing then this trainer argument is about to become your best friend.

The `fast_dev_run` argument in the trainer **runs 5 batches of training, validation, test and prediction** data through your trainer to see if there are any bugs:

```python
Trainer(fast_dev_run=True)

# Change to 7 batches:
Trainer(fast_dev_run=7)
```

> ⚠️ This argument will disable tuner, checkpoint callbacks, early stopping callbacks, loggers and logger callbacks like
> `LearningRateMonitor` and `DeviceStatsMonitor`.

### Shorten the epoch length

Sometimes it’s helpful to only use a fraction of your training, val, test, or predict data (or a set number of batches).
For example, you can use 20% of the training set and 1% of the validation set.

On larger datasets like Imagenet, this can help you debug or test a few things faster than waiting for a full epoch.

```python
# use only 10% of training data and 1% of val data
trainer = Trainer(limit_train_batches=0.1, limit_val_batches=0.01)  # NOTE.

# use 10 batches of train and 5 batches of val
trainer = Trainer(limit_train_batches=10, limit_val_batches=5)
```

### Run a Sanity Check

Lightning runs 2 steps of **validation in the beginning of training**. This avoids crashing in the validation loop
sometime deep into a lengthy training loop.

(See: `num_sanity_val_steps` argument of `Trainer`)

```python
trainer = Trainer(num_sanity_val_steps=2)
```

### Print LightningModule weights summary

Whenever the `.fit()` function gets called, the Trainer will print the weights summary for the `LightningModule`.

```python
trainer.fit(...)
```

```text
  | Name  | Type        | Params
----------------------------------
0 | net   | Sequential  | 132 K
1 | net.0 | Linear      | 131 K
2 | net.1 | BatchNorm1d | 1.0 K
```

[❗] To add the child modules to the summary add a `ModelSummary`:
```python
from pytorch_lightning.callbacks import ModelSummary

trainer = Trainer(callbacks=[ModelSummary(max_depth=-1)])  # NOTE!
```

To print the model summary if `.fit()` is not called:
```python
from pytorch_lightning.utilities.model_summary import ModelSummary

model = LitModel()
summary = ModelSummary(model, max_depth=-1)
print(summary)
```

To turn off the autosummary use:
```python
Trainer(enable_model_summary=False)
```

### Print input output layer dimensions

Another debugging tool is to display the intermediate input- and output sizes of all your layers **by setting the
`example_input_array` attribute** in your `LightningModule`.

```python
class LitModel(LightningModule):
    def __init__(self, *args, **kwargs):
        self.example_input_array = torch.Tensor(32, 1, 28, 28)  # NOTE!
```

With the input array, the summary table will include the input and output layer dimensions when you call 
`.fit()` on the Trainer.

```text
  | Name  | Type        | Params | In sizes  | Out sizes
--------------------------------------------------------------
0 | net   | Sequential  | 132 K  | [10, 256] | [10, 512]
1 | net.0 | Linear      | 131 K  | [10, 256] | [10, 512]
2 | net.1 | BatchNorm1d | 1.0 K  | [10, 512] | [10, 512]
```